In [ ]:
from keras.models import load_model
import keras.backend as K
import tensorflow as tf
import mtcnn
from mtcnn.mtcnn import MTCNN
import numpy as np
from numpy import asarray
from numpy import expand_dims
from numpy import linalg
from PIL import Image
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image_dir_basepath_train = './data/images_train'
image_dir_basepath_dev = './data/images_dev'
image_dir_basepath_test = './data/images_test'
image_size = 160
image_reduction_factor = 8

In [ ]:
model_path = './model/keras/model/facenet_keras.h5'
model = load_model(model_path)

In [ ]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

In [ ]:
def load_and_align_images(filepaths):
    detector = MTCNN()
    
    aligned_images_HR = []
    aligned_images_LR = []

    for filepath in filepaths:
        img = imread(filepath)

        pixels = asarray(img)
        results = detector.detect_faces(pixels)
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        
        image_HR = image.resize((image_size,image_size))
        aligned_HR = asarray(image_HR)      
        aligned_images_HR.append(aligned_HR)

        # Generate low-res through PIL 
        image_LR = image.resize((int(image_size/image_reduction_factor), int(image_size/image_reduction_factor)))
        aligned_LR = asarray(image_LR)
        aligned_images_LR.append(aligned_LR)

    return np.array(aligned_images_HR) , np.array(aligned_images_LR)

In [ ]:
def calc_embs(filepaths, batch_size=1):
    aligned_images_HR_b4prewhiten, aligned_images_LR_b4prewhiten = load_and_align_images(filepaths)
    aligned_images_HR = prewhiten(aligned_images_HR_b4prewhiten)
    aligned_images_LR = prewhiten(aligned_images_LR_b4prewhiten)

    pd_HR = []

    for start in range(0, len(aligned_images_HR), batch_size):
        pd_HR.append(model.predict_on_batch(aligned_images_HR[start:start+batch_size])) # Essentially passing one image at a time because batch_size = 1
    embs_HR = K.l2_normalize(np.concatenate(pd_HR), axis=1)
    embs_HR = K.eval(embs_HR)

    return aligned_images_HR_b4prewhiten, embs_HR, aligned_images_LR_b4prewhiten

In [ ]:
# Prepare training data
data = {}

image_dirpath = [os.path.join(image_dir_basepath_train, d) for d in os.listdir(image_dir_basepath_train)]

First = True
loopindex = 0

for d in image_dirpath: #Loop through each directory
    loopindex = loopindex+1
    print(loopindex)
    image_filepaths = [os.path.join(d,f) for f in os.listdir(d)] #create a list of file names in current directory
    aligned_images_HR_b4prewhiten, embs_HR, aligned_images_LR_b4prewhiten = calc_embs(image_filepaths)
    if First == True:
        X_train = aligned_images_LR_b4prewhiten
        X_train_HR = aligned_images_HR_b4prewhiten
        Y_train = embs_HR
        X_train_name = []
        for name_index in range(aligned_images_LR_b4prewhiten.shape[0]):
            X_train_name.append(d[20:])
        First = False
    else:
        X_train = np.append(X_train, aligned_images_LR_b4prewhiten, axis=0)
        X_train_HR = np.append(X_train_HR, aligned_images_HR_b4prewhiten, axis=0)
        Y_train = np.append(Y_train, embs_HR, axis=0)
        for name_index in range(aligned_images_LR_b4prewhiten.shape[0]):
            X_train_name.append(d[20:])

np.save('X_train.npy', X_train)
np.save('X_train_HR.npy', X_train_HR)
np.save('X_train_name.npy', X_train_name)
np.save('Y_train.npy', Y_train)

In [ ]:
# Prepare dev data
data = {}

image_dirpath = [os.path.join(image_dir_basepath_dev, d) for d in os.listdir(image_dir_basepath_dev)]

First = True
loopindex = 0

for d in image_dirpath: #Loop through each directory
    loopindex = loopindex+1
    print(loopindex)
    image_filepaths = [os.path.join(d,f) for f in os.listdir(d)] #create a list of file names in current directory
    aligned_images_HR_b4prewhiten, embs_HR, aligned_images_LR_b4prewhiten = calc_embs(image_filepaths)
    if First == True:
        X_dev = aligned_images_LR_b4prewhiten
        X_dev_HR = aligned_images_HR_b4prewhiten
        Y_dev = embs_HR
        X_dev_name = []
        for name_index in range(aligned_images_LR_b4prewhiten.shape[0]):
            X_dev_name.append(d[18:])
        First = False
    else:
        X_dev = np.append(X_dev, aligned_images_LR_b4prewhiten, axis=0)
        X_dev_HR = np.append(X_dev_HR, aligned_images_HR_b4prewhiten, axis=0)
        Y_dev = np.append(Y_dev, embs_HR, axis=0)
        for name_index in range(aligned_images_LR_b4prewhiten.shape[0]):
            X_dev_name.append(d[18:])

np.save('X_dev.npy', X_dev)
np.save('X_dev_HR.npy', X_dev_HR)
np.save('X_dev_name.npy', X_dev_name)
np.save('Y_dev.npy', Y_dev)

In [ ]:
# Prepare test data
data = {}

image_dirpath = [os.path.join(image_dir_basepath_test, d) for d in os.listdir(image_dir_basepath_test)]

First = True
loopindex = 0

for d in image_dirpath: #Loop through each directory
    loopindex = loopindex+1
    print(loopindex)
    image_filepaths = [os.path.join(d,f) for f in os.listdir(d)] #create a list of file names in current directory
    aligned_images_HR_b4prewhiten, embs_HR, aligned_images_LR_b4prewhiten = calc_embs(image_filepaths)
    if First == True:
        X_test = aligned_images_LR_b4prewhiten
        X_test_HR = aligned_images_HR_b4prewhiten
        Y_test = embs_HR
        X_test_name = []
        for name_index in range(aligned_images_LR_b4prewhiten.shape[0]):
            X_test_name.append(d[19:])
        First = False
    else:
        X_test = np.append(X_test, aligned_images_LR_b4prewhiten, axis=0)
        X_test_HR = np.append(X_test_HR, aligned_images_HR_b4prewhiten, axis=0)
        Y_test = np.append(Y_test, embs_HR, axis=0)
        for name_index in range(aligned_images_LR_b4prewhiten.shape[0]):
            X_test_name.append(d[19:])

np.save('X_test.npy', X_test)
np.save('X_test_HR.npy', X_test_HR)
np.save('X_test_name.npy', X_test_name)
np.save('Y_test.npy', Y_test)